<a href="https://colab.research.google.com/github/kamikater34/AutomaticTextSummarization/blob/master/NENA_script_PriceDownload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
import json
# this allows for more processing
import pandas as pd
from dateutil.parser import parse


API_TOKEN = None

API_LOGIN_ENDPOINT = "https://api.nena.no/api/user/login"
API_DATA_ENDPOINT = "https://api.nena.no/api/fundamental/series"

SERIES_PREFIX = 'athr' #Austrian Hourly series
SERIES_CODES = ['cons', 'wind', 'price'] #Consumption, Wind, Price


def main():
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application / json'}

    #STEP 1 Getting a authentication token from API
    LOGIN_PARAMS = {'UserName' : "kelag.struckl", 'Password' : "dlE8oSOHm4"}
    login_res = requests.post(url = API_LOGIN_ENDPOINT, data=json.dumps(LOGIN_PARAMS), headers = headers)
    login_res_dic = json.loads(login_res.text)
    API_TOKEN = login_res_dic["Token"]

    for ser_code in SERIES_CODES:
        #STEP 2 Getting Data using received token
        DATA_PARAMS= {
            "FromDateTime": "2021-05-05",  # or any date in yyyy-mm-dd format
            "ToDateTime": "today+15",  # today + 14 days ahead or any date ahead in yyyy-mm-dd format
            "Resolution": "hour",
            "SeriesId": SERIES_PREFIX + ser_code,
            "UserInfo": {
                "UserName": "kelag.struckl",
                "Token": API_TOKEN
            }
        }
        data_res = requests.post(url=API_DATA_ENDPOINT, data=json.dumps(DATA_PARAMS), headers=headers).json()
        
        #parse dates in the Values dictionary
        parse_data_res = {parse(k): float(v) for k,v in data_res["Values"].items()}
        
        #make a pandas dataframe 
        df = pd.DataFrame(list(parse_data_res.items()))
        df.columns = ['Date', 'AUSTRIA '+ ser_code.upper()]
        df = df.set_index('Date') 
        
        #Create an excel file per series code from dataframe
        df.to_excel('AT_'+ser_code+ '.xls', )

if __name__ == '__main__':
    main()


In [4]:
!pwd


/content
